In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import pandas as pd

import random

import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, TimeDistributed, RepeatVector
from keras.optimizers import Adam
from lib.township_range import TownshipRanges
from lib.read_data import read_and_join_output_file
from lib.deeplearning import get_train_test_datasets,  get_sets_shapes, evaluate_forecast, get_data_for_prediction, combine_all_target_years, get_year_to_year_differences
from lib.viz import view_trs_side_by_side, simple_geodata_viz

In [3]:
RANDOM_SEED = 31
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tensorflow.random.set_seed(RANDOM_SEED)

In [4]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

Num GPUs Available:  0


## Preparing the Dataset
### The Train-Test Split
The dataset is made of 478 Township-Ranges, each containing a multivariate (81 features) time series (data between 2014 to 2021). This dataset can thus be seen as a 3 dimensional dataset of
$478 TownshipRanges * 8 time stamps * 81 features$
The objective is to predict the 2022 target value of `GSE_GWE` (Ground Surface Elevation to Groundwater Water Elevation - Depth to groundwater elevation in feet below ground surface) for each Township-Range.

LSTM neural networks can be used for time series forecasting and take inputs of the shape *[samples, time series steps, features]*. This perfectly fits our dataset.
To fit our dataset and objective, as well as LSTM neural networks architecture we will thus perform the train test split as follow:
* Training and Test sets will be split by Township-Ranges. I.e., some Township-Ranges will have all their 2014-2021 data points in the training set, some others will be in the the test set.
* The model will be trained based on the 2014-2020 data for all features - including the target feature - and will be trained and tested on the 2021 value of the target feature.

With such a method, unlike a simple time series forecasting where the target feature is forecasted only based on its past value, we allow past value of other features (in our case cultivated crops, precipitations, population density, number of wells drilled) to influence the future value of the target feature.

![Train-Test Split](../doc/images/deeplearning-train-test-split.jpg)

We do not create a validation dataset as we use Keras internal cross-validation mechanism to shuffle the data points (i.e., the Township-Ranges) and keep some for the validation at each training epoch.
### Data Imputation and Scaling
Missing data imputation for a Township-Range is performed only using the existing data of that Township-Range (not the data of all Township-Ranges). For example:
* a *fill forward* approach is used for many fields like crops, vegetation and soils. The percentage of land use per crop in 2014 in a Township-Range is imputed into the missing year 2015 for that particular Township-Range.
* for fields like `PCT_OF_CAPACITY` (the capacity percentage of water reservoir), missing values in a Township-Range are filled using the min, mean, median or max values of that particular Township-Range
This approach means that the data imputation *fit* method does not need to learn values from other Township-Ranges data points to impute missing values. Since our train and test datasets are split by Township-Ranges, it avoids issues when  using the impute pipeline fitted on the training dataset to impute data for Township-Ranges the impute pipeline has not seen before.

We use a MinMax scaler to scale all values between 0 and 1 for the neural network.

It should be noted that we do not need to do any data imputation on the training and test sets *y* target feature since it does not have any missing data point.

In [5]:
test_size=0.15
target_variable="GSE_GWE"
# Load the data from the ETL output files
X = read_and_join_output_file()
# Split the input pandas Dataframe into training and test datasets, applies the impute pipeline
# transformation and reshapes the datasets to 3D (samples, time, features) numpy arrays
X_train, X_test, y_train, y_test, impute_pipeline, target_scaler = get_train_test_datasets(X, target_variable=target_variable,
    test_size=test_size, random_seed=RANDOM_SEED)
model_predictions_df = pd.DataFrame(y_test, columns=[target_variable])
model_scores_df = pd.DataFrame(columns=["mae", "mse", "rmse"])
nb_features = X_train.shape[-1]
get_sets_shapes(X_train, X_test)

,nb_items,nb_timestamps,nb_features
training dataset,406,7,81
test dataset,72,7,81


## Training Different Models
We tried 3 different LSTM models:
* A simple model made of a single *LSTM* layer and an output *Dense* layer
* A model made of a *LSTM* layer followed by a *Dense* and *Dropout* layers before the output layer
* An Encoder-Decoder model made of 2 *LSTM* layers followed by a *Dense* and *Dropout* layers

![LSTM Model Architectures](../doc/images/deeplearning-architectures.jpg)


Encoder-decoder architectures are more common for sequence to sequence learning e.g., when forecasting the next 3 days (output sequence of length 3) based on the past year data (input sequence of length 365). In our case we only predict data for 1 time step in the feature. The output sequence being of length 1 this architecture might seem superfluous but has been tested anyway. This architecture was inspired by the Encoder-Decoder architecture in this article: *[CNN-LSTM-Based Models for Multiple Parallel Input and Multi-Step Forecast](https://towardsdatascience.com/cnn-lstm-based-models-for-multiple-parallel-input-and-multi-step-forecast-6fe2172f7668)*.

As such models are made for sequence to sequence learning and forecasting, the output of such a model is different from the previous ones. It has an output of size *[samples, forcasting sequence length, target features]*. In our case the forecasting sequence length and number of target features are both 1.

## Training Model 1 - Simple LSTM Model

In [6]:
m1_hyper_parameters = {
    "lstm_units": 160,
    "lstm_activation": "sigmoid",
    "learning_rate": 0.001,
    "validation_split": 0.1,
    "batch_size": 128,
    "epochs": 270,
}

model1 = Sequential()
model1.add(LSTM(m1_hyper_parameters["lstm_units"], activation=m1_hyper_parameters["lstm_activation"], input_shape=(7, nb_features)))
model1.add(Dense(1, activation="linear"))
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 160)               154880    
                                                                 
 dense (Dense)               (None, 1)                 161       
                                                                 
Total params: 155,041
Trainable params: 155,041
Non-trainable params: 0
_________________________________________________________________


In [7]:
model1.compile(loss="mse", optimizer=Adam(learning_rate=m1_hyper_parameters["learning_rate"]), metrics=[keras.metrics.RootMeanSquaredError()])
model1.fit(X_train, y_train,
           validation_split=m1_hyper_parameters["validation_split"],
           batch_size=m1_hyper_parameters["batch_size"],
           epochs=m1_hyper_parameters["epochs"],
           shuffle=True)
yhat = model1.predict(X_test, verbose=0)
yhat_inverse = target_scaler.inverse_transform(yhat)
model_predictions_df["model_1_prediction"] = yhat_inverse
model_scores_df.loc["model 1"] = evaluate_forecast(y_test, yhat_inverse)

Epoch 1/270
3/3 [==============================] - 1s 143ms/step - loss: 0.1125 - root_mean_squared_error: 0.3354 - val_loss: 0.1359 - val_root_mean_squared_error: 0.3686
Epoch 2/270
3/3 [==============================] - 0s 21ms/step - loss: 0.0867 - root_mean_squared_error: 0.2945 - val_loss: 0.0269 - val_root_mean_squared_error: 0.1641
Epoch 3/270
3/3 [==============================] - 0s 23ms/step - loss: 0.0568 - root_mean_squared_error: 0.2384 - val_loss: 0.0396 - val_root_mean_squared_error: 0.1990
Epoch 4/270
3/3 [==============================] - 0s 22ms/step - loss: 0.0579 - root_mean_squared_error: 0.2406 - val_loss: 0.0281 - val_root_mean_squared_error: 0.1676
Epoch 5/270
3/3 [==============================] - 0s 22ms/step - loss: 0.0434 - root_mean_squared_error: 0.2082 - val_loss: 0.0568 - val_root_mean_squared_error: 0.2383
Epoch 6/270
3/3 [==============================] - 0s 22ms/step - loss: 0.0466 - root_mean_squared_error: 0.2158 - val_loss: 0.0306 - val_root_mean_s

## Training Model 2 - LSTM + Dense Layer Model

In [100]:
m2_hyper_parameters = {
    "lstm_units": 100,
    "lstm_activation": "sigmoid",
    "dense_units": 11,
    "dense_activation": "tanh",
    "dropout": 0.1,
    "learning_rate": 0.0001,
    "validation_split": 0.1,
    "batch_size": 32,
    "epochs": 200,
}

model2 = Sequential()
model2.add(LSTM(m2_hyper_parameters["lstm_units"], activation=m2_hyper_parameters["lstm_activation"], input_shape=(7, nb_features)))
model2.add(Dense(m2_hyper_parameters["dense_units"], activation=m2_hyper_parameters["dense_activation"]))
model2.add(Dropout(m2_hyper_parameters["dropout"]))
model2.add(Dense(1, activation="linear"))
model2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 100)               72800     
                                                                 
 dense_13 (Dense)            (None, 11)                1111      
                                                                 
 dropout_4 (Dropout)         (None, 11)                0         
                                                                 
 dense_14 (Dense)            (None, 1)                 12        
                                                                 
Total params: 73,923
Trainable params: 73,923
Non-trainable params: 0
_________________________________________________________________


In [101]:
model2.compile(loss="mse", optimizer=Adam(learning_rate=m2_hyper_parameters["learning_rate"]), metrics=[keras.metrics.RootMeanSquaredError()])
model2.fit(X_train, y_train,
           validation_split=m2_hyper_parameters["validation_split"],
           batch_size=m2_hyper_parameters["batch_size"],
           epochs=m2_hyper_parameters["epochs"],
           shuffle=True)
yhat = model2.predict(X_test, verbose=0)
yhat_inverse = target_scaler.inverse_transform(yhat)
model_predictions_df["model_2_prediction"] = yhat_inverse
model_scores_df.loc["model 2"] = evaluate_forecast(y_test, yhat_inverse)

Epoch 1/200
12/12 [==============================] - 1s 25ms/step - loss: 1.0335 - root_mean_squared_error: 1.0166 - val_loss: 0.8031 - val_root_mean_squared_error: 0.8962
Epoch 2/200
12/12 [==============================] - 0s 7ms/step - loss: 0.5740 - root_mean_squared_error: 0.7577 - val_loss: 0.3972 - val_root_mean_squared_error: 0.6302
Epoch 3/200
12/12 [==============================] - 0s 7ms/step - loss: 0.2602 - root_mean_squared_error: 0.5101 - val_loss: 0.1709 - val_root_mean_squared_error: 0.4135
Epoch 4/200
12/12 [==============================] - 0s 7ms/step - loss: 0.1250 - root_mean_squared_error: 0.3536 - val_loss: 0.0700 - val_root_mean_squared_error: 0.2646
Epoch 5/200
12/12 [==============================] - 0s 7ms/step - loss: 0.0695 - root_mean_squared_error: 0.2636 - val_loss: 0.0347 - val_root_mean_squared_error: 0.1864
Epoch 6/200
12/12 [==============================] - 0s 7ms/step - loss: 0.0649 - root_mean_squared_error: 0.2547 - val_loss: 0.0249 - val_root_

## Training Model 3 - Encoder-Decoder LSTM Model

In [102]:
m3_hyper_parameters = {
    "lstm_units": 300,
    "lstm_activation": "sigmoid",
    "2nd_lstm_units": 140,
    "2nd_lstm_activation": "sigmoid",
    "dense_units": 21,
    "dense_activation": "tanh",
    "dropout": 0.2,
    "learning_rate": 0.001,
    "validation_split": 0.1,
    "batch_size": 32,
    "epochs": 200,
}


model3 = Sequential()
model3.add(LSTM(m3_hyper_parameters["lstm_units"], activation=m3_hyper_parameters["lstm_activation"], input_shape=(7, nb_features)))
model3.add(RepeatVector(1))
model3.add(LSTM(m3_hyper_parameters["2nd_lstm_units"], activation=m3_hyper_parameters["lstm_activation"], return_sequences=True))
model3.add(TimeDistributed(Dense(m3_hyper_parameters["dense_units"], activation=m3_hyper_parameters["dense_activation"])))
model3.add(Dropout(m3_hyper_parameters["dropout"]))
model3.add(Dense(1, activation="linear"))
model3.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 300)               458400    
                                                                 
 repeat_vector_2 (RepeatVect  (None, 1, 300)           0         
 or)                                                             
                                                                 
 lstm_13 (LSTM)              (None, 1, 140)            246960    
                                                                 
 time_distributed_2 (TimeDis  (None, 1, 21)            2961      
 tributed)                                                       
                                                                 
 dropout_5 (Dropout)         (None, 1, 21)             0         
                                                                 
 dense_16 (Dense)            (None, 1, 1)            

In [103]:
y_train_3d =  y_train[..., np.newaxis]
model3.compile(loss="mse", optimizer=Adam(learning_rate=m3_hyper_parameters["learning_rate"]), metrics=[keras.metrics.RootMeanSquaredError()])
model3.fit(X_train, y_train_3d,
           validation_split=m3_hyper_parameters["validation_split"],
           batch_size=m3_hyper_parameters["batch_size"],
           epochs=m3_hyper_parameters["epochs"],
           shuffle=True)
yhat = model3.predict(X_test, verbose=0)
yhat_inverse = target_scaler.inverse_transform(yhat.squeeze(2))
model_predictions_df["model_3_prediction"] = yhat_inverse
model_scores_df.loc["model 3"] = evaluate_forecast(y_test, yhat_inverse)

Epoch 1/200
12/12 [==============================] - 2s 44ms/step - loss: 0.3372 - root_mean_squared_error: 0.5807 - val_loss: 0.1492 - val_root_mean_squared_error: 0.3863
Epoch 2/200
12/12 [==============================] - 0s 20ms/step - loss: 0.1210 - root_mean_squared_error: 0.3478 - val_loss: 0.0275 - val_root_mean_squared_error: 0.1658
Epoch 3/200
12/12 [==============================] - 0s 19ms/step - loss: 0.0659 - root_mean_squared_error: 0.2567 - val_loss: 0.0269 - val_root_mean_squared_error: 0.1641
Epoch 4/200
12/12 [==============================] - 0s 19ms/step - loss: 0.0554 - root_mean_squared_error: 0.2354 - val_loss: 0.0283 - val_root_mean_squared_error: 0.1682
Epoch 5/200
12/12 [==============================] - 0s 19ms/step - loss: 0.0508 - root_mean_squared_error: 0.2254 - val_loss: 0.0251 - val_root_mean_squared_error: 0.1584
Epoch 6/200
12/12 [==============================] - 0s 19ms/step - loss: 0.0480 - root_mean_squared_error: 0.2191 - val_loss: 0.0273 - val_

## Comparing the Different Models
### Comparing the Model Scores

In [104]:
model_scores_df

,mae,mse,rmse
model 1,23.433969,1199.347168,34.631592
model 2,43.358646,3773.487793,61.428722
model 3,35.156990,1966.765625,44.348232


### Comparing the Model Predictions
Here we are comparing the target variable values for the year 2021 for the Township-Ranges in the test set compared to the prediction made by each model based on the 2014-2020 data for the Township-Ranges in the test set.

In [105]:
model_predictions_df

,GSE_GWE,model_1_prediction,model_2_prediction,model_3_prediction
0,33.198000,24.707844,43.789276,3.123997
1,34.795000,50.173042,60.214745,26.909847
2,161.756667,65.959122,52.021770,37.934628
3,54.423000,42.324570,60.615677,16.992435
4,80.653077,96.485344,136.194901,66.975838
...,...,...,...,...
67,187.252308,181.199005,179.748108,157.927078
68,179.551290,153.865189,201.877457,136.810684
69,236.543750,249.583542,241.438934,233.151688
70,292.550000,273.464783,201.449844,248.122894


Based on the model scores it turns out that the simplest of the three LSTM models is the one having the best scores.

However the `GSE_GWE` (Ground Surface Elevation to Groundwater Water Elevation - Depth to groundwater elevation in feet below ground surface) target value has a median of 135.47 (~41.3 meters) and a mean value of 166.17 feet (~50.6 meters). A mean average error of 23.66 feet (7.2 meters), and root mean square error of 34.82 feet (10.6 meters) in the prediction is fairly large. Even the best model is not accurate enough to be useful.
## Predicting 2022
Even though our best model has a too large error to be useful, we can try as an exercise, to predict the 2022 target variable for all the Township-Ranges.

The model was train to predict the 2021 data based on the previous 7 years of data 2014 to 2020. To predict 2022 we thus need to pass the previous 7 years of data (2015-2021). To do so:
1. We use our impute pipeline trained on the training dataset to impute values on the entire dataset and normalize the data
2. We drop the 2014 data points
3. We reshape the dataset as a 3 dimensional numpy array in the form of [all Township-Ranges, 2015-2021, 81 features]

Once we predict the 2022 values of the target variable, we extract the 2021 values from the original dataset to compare the 2021 values with the rpedicted 2022 values

In [8]:
# Predict the 2022 values for all Township-Ranges for the target variable based on 2015-2021 data
X_2015_to_2021 = get_data_for_prediction(X, impute_pipeline)
yhat_2022 = model1.predict(X_2015_to_2021, verbose=0)
yhat_inverse_2022 = target_scaler.inverse_transform(yhat_2022)
predictions_2022_df = pd.DataFrame(yhat_inverse_2022, index=X.index.get_level_values(0).unique(), columns=[target_variable])
# Add the 2022 values of the target variable to the existing ones
all_years_df = combine_all_target_years(X, target_variable, predictions_2022_df)
all_years_df

GSE_GWE
TOWNSHIP_RANGE YEAR            
T01N R02E      2014   57.046154
               2015   56.027436
               2016   48.830000
               2017   48.007333
               2018   45.985000
...                         ...
T32S R30E      2018  405.450000
               2019  413.150000
               2020  404.600000
               2021  383.500000
               2022  382.509857

[4302 rows x 1 columns]

In [ ]:
township_range = TownshipRanges()
all_years_map_df = pd.merge(township_range.sjv_township_range_df, township_range.reset_index(), how="left", on=["TOWNSHIP_RANGE", ])
view_trs_side_by_side(all_years_map_df, feature="YEAR", value="GSE_GWE", title="San Joaquin Valley GSE_GWE with 2022 predictions")

We can also compare the year to year difference in the target variable from 2014 to 2021 and between our 2022 predictions and the 2021 values.

In [9]:
yty_difference_df = get_year_to_year_differences(X, target_variable, predictions_2022_df)
yty_difference_df

,2014_2015,2015_2016,2016_2017,2017_2018,2018_2019,2019_2020,2020_2021
TOWNSHIP_RANGE,,,,,,,
T01N R02E,-1.018718,-7.197436,-0.822667,-2.022333,-0.203500,6.414500,0.997636
T01N R03E,5.538801,-10.782982,-1.871021,-1.744653,0.230496,0.134899,8.257401
T01N R04E,14.321818,-4.161818,-2.347333,3.394000,-2.202381,4.957381,-2.288810
T01N R05E,8.593333,-3.834333,-1.543286,2.161978,1.180974,4.827487,-0.859790
T01N R06E,0.445625,6.296518,-8.221518,2.960375,-0.574158,11.043158,0.818000
...,...,...,...,...,...,...,...
T32S R26E,-5.275000,4.211538,-26.840110,8.766807,0.764396,11.478138,23.135897
T32S R27E,6.091629,9.723077,17.857692,-8.739271,-0.926316,-19.604605,32.741071
T32S R28E,-42.664167,51.418452,54.644048,-79.904487,20.433654,-0.816071,-17.148352


As listed above we then extract the 2015 to 2021 data for all Township-Ranges, predict the 2022 values using our best model and compare the results.

In [22]:
difference_df = pd.merge(township_range.sjv_township_range_df, pd.melt(yty_difference_df.reset_index(), id_vars=["TOWNSHIP_RANGE"], var_name="YEAR", value_name="GSE_GWE_DIFFERENCE"), how="left", on=["TOWNSHIP_RANGE", ])
view_trs_side_by_side(difference_df, feature="YEAR", value="GSE_GWE_DIFFERENCE", title="San Joaquin Valley GSE_GWE year-to-year variations from 2014 until 2022 predictions")

alt.Chart(...)

In [ ]:
yty_difference_df.describe()

## Conclusion
Using a simple LSTM neural network to make next year predictions based on the past 7 years of data, we are able to achieve a more accurate prediction on the test set with an RMSE of 34.82 feet (10.6 meters) compared to an RMSE between 75 and 95 feet (22.8 and 28.9 meters) using supervised algorithms like XGBoost or K-Neighbours regressor.

Yet the predictions have a far too high error rate to be reliable and useful.